In [145]:
# Import necessary libraries
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd
import nltk
from nltk import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import joblib
# Download NLTK resources
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def load_data(file_path):
    # Load data from CSV file
    df = pd.read_csv(file_path)
    return df

def split_data(feature,labels):
    # Split the data into training and testing sets
    sentences = feature
    label = labels
    cleaned_text = preprocess_text(sentences)
    
    clean_converted = convert_input(cleaned_text)
    
    x_train, x_test, y_train, y_test = train_test_split(clean_converted, label, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test, tokenizer

def preprocess_text(sentences):
    # Perform necessary preprocessing steps like tokenization, stopword removal, stemming, lemmatization, etc.
    cleaned_text = []
    for text in sentences:
        cleaned_text.append(preprocess_single_text(text))
    return cleaned_text

def preprocess_single_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove all tokens which are non-alphabetic
    words = [word for word in tokens if word.isalpha()]
    # Lowercase all words
    words = [word.lower() for word in words]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Lemmatize all words into a new list
    lemmatizer = nltk.stem.WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Return a list of words
    return ' '.join(words)

def build_model(tokenizer):
    # Build the model
    embedding_dim = 16

    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=20))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(model, x_train, y_train, x_test, y_test):
    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=2, validation_data=(x_test, y_test))

def evaluate_model(model, x_test, y_test):
    # Evaluate the model
    loss, accuracy = model.evaluate(x_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

def predict_and_print_labels(model, some_input, tokenizer):
    # Predict labels for test sentences and print the results
    converted_input = convert_input(some_input)
    predictions = model.predict(converted_input)

    for i in range(len(test_sentences)):
        label = "Positive" if predictions[i] > 0.5 else "Negative"
        print(f"Text: {test_sentences[i]}, Predicted label: {label}")
    
    return predictions

    
def convert_input(test_sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(test_sentences)
    sequences = tokenizer.texts_to_sequences(test_sentences)
    padded_sequences = pad_sequences(sequences, maxlen=20, padding='post', truncating='post')

    return padded_sequences


def save_model(model, filename):
    # Save the model to a file
    model.save(filename)
    print(f"Model saved to {filename}")

def load_saved_model(filename):
    # Load a saved model from a file
    loaded_model = tf.keras.models.load_model(filename)
    print(f"Model loaded from {filename}")
    return loaded_model

def create_binary_model(file_path,label_col):
    df = load_data(file_path)
    features = df['Obs']
    labels = df[label_col]
    x_train, x_test, y_train, y_test,tokenizer = split_data(features,labels)
    model = build_model(tokenizer)
    train_model(model, x_train, y_train, x_test, y_test)
    evaluate_model(model, x_test, y_test)
    save_model(model, f'trained_model_{label_col}.h5')
# loaded_model = load_saved_model('trained_model1.h5')

def create_testers(df,label_colummn,x,y):
    test_sentences = df['Obs'][x:y].tolist()
    test_labels = df[label_colummn][x:y]
    all_labels = df[['A1','A2','A3','B1','B2','B3','B4']][x:y]

    return test_sentences,test_labels,all_labels



# Calculate precision, recall, and F1-score
def calculate_metrics(y_true, y_pred):
    precision, recall, f1_score, support = precision_recall_fscore_support(y_true, y_pred)
    return precision, recall, f1_score

def save_results(predictions, precision, recall, f1_score, filename):
    with open(filename, "wb") as f:
        joblib.dump({"predictions": predictions, "precision": precision, "recall": recall, "f1_score": f1_score}, f)

def predict_ensemble(models, X):
    predictions = []
    for model in models:
   
        predictions.append(model.predict(X))

  # Take the average of the predictions from each model.
    

    return predictions

def convert_predictions_to_binary(predictions):
    # Convert predictions to binary
    binary_predictions = []
    for prediction in predictions:
        binary_prediction = []
        for value in prediction:
            if value > 0.5:
                binary_prediction.append(1)
            else:
                binary_prediction.append(0)
        binary_predictions.append(binary_prediction)
    return binary_predictions

    
create_binary_model('Problem_Dataset.csv','A1')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Manoj Patil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Manoj
[nltk_data]     Patil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/10
420/420 [==============================] - 2s 3ms/step - loss: 0.4219 - accuracy: 0.8548 - val_loss: 0.3464 - val_accuracy: 0.8619
Epoch 2/10
420/420 [==============================] - 1s 3ms/step - loss: 0.2650 - accuracy: 0.8762 - val_loss: 0.2386 - val_accuracy: 0.9000
Epoch 3/10
420/420 [==============================] - 1s 3ms/step - loss: 0.1408 - accuracy: 0.9512 - val_loss: 0.2259 - val_accuracy: 0.9048
Epoch 4/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0648 - accuracy: 0.9869 - val_loss: 0.2384 - val_accuracy: 0.9000
Epoch 5/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0293 - accuracy: 0.9952 - val_loss: 0.2661 - val_accuracy: 0.9095
Epoch 6/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0122 - accuracy: 0.9988 - val_loss: 0.2986 - val_accuracy: 0.9143
Epoch 7/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.3134 - val_accuracy: 0.9095
Epoch 

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [146]:

create_binary_model('Problem_Dataset.csv','A2')
create_binary_model('Problem_Dataset.csv','A3')
create_binary_model('Problem_Dataset.csv','B1')
create_binary_model('Problem_Dataset.csv','B2')
create_binary_model('Problem_Dataset.csv','B3')
create_binary_model('Problem_Dataset.csv','B4')

Epoch 1/10
420/420 [==============================] - 2s 3ms/step - loss: 0.4180 - accuracy: 0.8512 - val_loss: 0.2968 - val_accuracy: 0.8762
Epoch 2/10
420/420 [==============================] - 1s 3ms/step - loss: 0.2019 - accuracy: 0.9095 - val_loss: 0.1660 - val_accuracy: 0.9619
Epoch 3/10
420/420 [==============================] - 1s 2ms/step - loss: 0.1005 - accuracy: 0.9643 - val_loss: 0.1605 - val_accuracy: 0.9571
Epoch 4/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0434 - accuracy: 0.9917 - val_loss: 0.1708 - val_accuracy: 0.9476
Epoch 5/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0203 - accuracy: 0.9964 - val_loss: 0.1848 - val_accuracy: 0.9333
Epoch 6/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.2165 - val_accuracy: 0.9429
Epoch 7/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.2261 - val_accuracy: 0.9286
Epoch 

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
420/420 [==============================] - 2s 3ms/step - loss: 0.4253 - accuracy: 0.8536 - val_loss: 0.3700 - val_accuracy: 0.8524
Epoch 2/10
420/420 [==============================] - 1s 2ms/step - loss: 0.2399 - accuracy: 0.8917 - val_loss: 0.2230 - val_accuracy: 0.9238
Epoch 3/10
420/420 [==============================] - 1s 2ms/step - loss: 0.1203 - accuracy: 0.9536 - val_loss: 0.2147 - val_accuracy: 0.9333
Epoch 4/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0701 - accuracy: 0.9774 - val_loss: 0.2166 - val_accuracy: 0.9286
Epoch 5/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0379 - accuracy: 0.9917 - val_loss: 0.2244 - val_accuracy: 0.9429
Epoch 6/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0208 - accuracy: 0.9976 - val_loss: 0.2405 - val_accuracy: 0.9333
Epoch 7/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0121 - accuracy: 0.9988 - val_loss: 0.2578 - val_accuracy: 0.9333
Epoch 

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
420/420 [==============================] - 2s 4ms/step - loss: 0.3565 - accuracy: 0.8631 - val_loss: 0.2303 - val_accuracy: 0.8333
Epoch 2/10
420/420 [==============================] - 1s 3ms/step - loss: 0.1138 - accuracy: 0.9583 - val_loss: 0.1436 - val_accuracy: 0.9524
Epoch 3/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0271 - accuracy: 0.9952 - val_loss: 0.1428 - val_accuracy: 0.9476
Epoch 4/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.1375 - val_accuracy: 0.9619
Epoch 5/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.1569 - val_accuracy: 0.9524
Epoch 6/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1538 - val_accuracy: 0.9619
Epoch 7/10
420/420 [==============================] - 1s 2ms/step - loss: 9.8317e-04 - accuracy: 1.0000 - val_loss: 0.1633 - val_accuracy: 0.9619
Ep

...

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
420/420 [==============================] - 2s 3ms/step - loss: 0.3888 - accuracy: 0.8643 - val_loss: 0.3301 - val_accuracy: 0.8286
Epoch 2/10
420/420 [==============================] - 1s 2ms/step - loss: 0.1832 - accuracy: 0.8881 - val_loss: 0.1608 - val_accuracy: 0.9381
Epoch 3/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0443 - accuracy: 0.9917 - val_loss: 0.1127 - val_accuracy: 0.9476
Epoch 4/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.1069 - val_accuracy: 0.9476
Epoch 5/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.1077 - val_accuracy: 0.9524
Epoch 6/10
420/420 [==============================] - 1s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1138 - val_accuracy: 0.9524
Epoch 7/10
420/420 [==============================] - 1s 3ms/step - loss: 8.2637e-04 - accuracy: 1.0000 - val_loss: 0.1173 - val_accuracy: 0.9524
Ep

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
420/420 [==============================] - 2s 2ms/step - loss: 0.4184 - accuracy: 0.8500 - val_loss: 0.2864 - val_accuracy: 0.8714
Epoch 2/10
420/420 [==============================] - 1s 2ms/step - loss: 0.1562 - accuracy: 0.9179 - val_loss: 0.0868 - val_accuracy: 0.9667
Epoch 3/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0294 - accuracy: 0.9964 - val_loss: 0.0667 - val_accuracy: 0.9762
Epoch 4/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0085 - accuracy: 0.9988 - val_loss: 0.0566 - val_accuracy: 0.9762
Epoch 5/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0614 - val_accuracy: 0.9762
Epoch 6/10
420/420 [==============================] - 1s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0676 - val_accuracy: 0.9714
Epoch 7/10
420/420 [==============================] - 1s 3ms/step - loss: 9.9576e-04 - accuracy: 1.0000 - val_loss: 0.0604 - val_accuracy: 0.9762
Ep

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [147]:
# Example usage for prediction
test_sentences,test_labels,all_labelss = create_testers(df,'A1',0,10)

loaded_model = load_saved_model('trained_model_A1.h5')
predeictionsss = predict_and_print_labels(loaded_model, test_sentences, tokenizer)
print(predeictionsss)
binary_predeictionsss = convert_predictions_to_binary(predeictionsss)

print(binary_predeictionsss)
print(test_labels)
print(all_labelss)
precision, recall, f1_score = calculate_metrics(test_labels, binary_predeictionsss)

print(f"Precision: {precision}, Recall: {recall}, F1-score: {f1_score}")

save_results(binary_predeictionsss, precision, recall, f1_score, "results.pkl")




Model loaded from trained_model_A1.h5
1/1 [==============================] - 0s 64ms/step
Text: Observed child fixated on a particular texture, rubbing a piece of sandpaper continuously throughout the appointment., Predicted label: Negative
Text: Patient's focus centers on vacuum cleaners, studying different models and their components., Predicted label: Negative
Text: Displays a strong interest in smelling various objects, often lingering on scents for prolonged periods., Predicted label: Negative
Text: Patient's attachment to a specific book is evident, quoting passages frequently., Predicted label: Negative
Text: Limited awareness of personal boundaries, invades others' personal space., Predicted label: Positive
Text: Repeatedly opens and closes doors, seemingly fascinated by the motion of hinges., Predicted label: Negative
Text: Engages in repetitive smelling and touching of objects in her environment., Predicted label: Negative
Text: Has difficulties in understanding social cues a

c:\Users\Manoj Patil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [148]:

test1,test2,test3 = create_testers(df,'A1',1,2)
converted_test1 = convert_input(test1)
print(test3)
print(converted_test1)
ensemble_predictions = predict_ensemble(models, converted_test1)
binary_ensemble_predictions = convert_predictions_to_binary(ensemble_predictions)

print(ensemble_predictions)
print(binary_ensemble_predictions)

ensemble_precision, ensemble_recall, ensemble_f1_score = calculate_metrics(test3, ensemble_predictions)
print(f"Precision: {ensemble_precision}, Recall: {ensemble_recall}, F1-score: {ensemble_f1_score}")


   A1  A2  A3  B1  B2  B3  B4
1   0   0   0   0   0   1   0
[[ 1  2  3  4  5  6  7  8  9 10 11 12  0  0  0  0  0  0  0  0]]
1/1 [==============================] - 0s 21ms/step
[array([[0.32294694]], dtype=float32), array([[2.513604e-05]], dtype=float32), array([[0.5854795]], dtype=float32), array([[0.00030022]], dtype=float32), array([[1.231672e-05]], dtype=float32), array([[0.09366304]], dtype=float32), array([[0.00017045]], dtype=float32)]
[[0], [0], [1], [0], [0], [0], [0]]


ValueError: Found input variables with inconsistent numbers of samples: [1, 7]

In [ ]:
test1,test2,test3 = create_testers(df,'A3',0,1)
converted_test1 = convert_input(test1)

In [ ]:
print(converted_test1)

[[ 2  3  4  5  1  6  7  8  1  9 10 11 12 13 14 15  0  0  0  0]]


In [ ]:
ensemble_predictions = predict_ensemble(models, converted_test1)

print(ensemble_predictions)

1/1 [==============================] - 0s 21ms/step
[array([[0.07126191]], dtype=float32), array([[0.00241495]], dtype=float32), array([[0.9489908]], dtype=float32), array([[0.00231021]], dtype=float32), array([[3.33789e-05]], dtype=float32), array([[0.39199013]], dtype=float32), array([[0.04412718]], dtype=float32)]
